<a href="https://colab.research.google.com/github/freezer2019/Analytics/blob/main/Stageavancement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geemap

In [2]:
import ee
import geemap
import pandas as pd
import numpy as np
import math
import datetime as dt
from numpy import log as ln
from sklearn.model_selection import train_test_split
from google.colab import drive 

In [3]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=xPi5d2X8mp_UsQZDu264TBPELxnHxpokhj_S90bK7ts&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjXYZiif9y80uvR3ivwI7cKZSS-8Hmmf3U9PTizSi8xak0VzLJtlr4

Successfully saved authorization token.


In [5]:
ee.Initialize()

In [6]:
Geometrysite1 =ee.Geometry.Polygon([[[-7.762724399566642,32.140949340957775], [-7.763453960418693,32.14083124166395],
                                 [-7.763298392295829,32.140399723713095], [-7.762584924697868,32.14051328126656],
                                 [-7.762724399566642,32.140949340957775]]])

Geometrysite2 =ee.Geometry.Polygon([[[-7.7617802619934,32.14051385752291], [-7.76253128051757,32.140341249986854],
                                     [-7.762440085411064,32.140032372528], [-7.761667609214775,32.14021860759168],
                                     [-7.7617802619934,32.14051385752291]]])

Geometrysite3 =ee.Geometry.Polygon([[[-7.763303756713859,32.14031796218702], [-7.763207197189323,32.13990915345722],
                                     [-7.762488365173332,32.14002725394517], [-7.762563467025749,32.14041789293111],
                                     [-7.763303756713859,32.14031796218702]]])


In [7]:
def addmaskL8sr(image):

  cloudShadowBitMask = (1 << 3);
  cloudsBitMask = (1 << 5);
  qa = image.select('pixel_qa');
  mask1 = qa.bitwiseAnd(cloudShadowBitMask).eq(0)
  mask2 = mask1.bitwiseAnd(cloudsBitMask).eq(0)
  return image.updateMask(mask2)

In [8]:
satdataset1 = (ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
              .filterBounds (Geometrysite1)
              .filter(ee.Filter.date('2020-04-01', '2021-03-31'))
              .map(addmaskL8sr)
              .map(lambda imag: imag.clip(Geometrysite1)))

landsurfacedata1 = (ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')
                   .filterBounds (Geometrysite1)
                   .filter(ee.Filter.date('2020-04-01', '2021-03-31'))
                  #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))
                   .map(lambda imag: imag.clip(Geometrysite1)))


satdataset2 = (ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
              .filterBounds (Geometrysite2)
              .filter(ee.Filter.date('2020-04-01', '2021-03-31'))
              .map(addmaskL8sr)
              .map(lambda imag: imag.clip(Geometrysite2)))

landsurfacedata2 = (ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')
                   .filterBounds (Geometrysite2)
                   .filter(ee.Filter.date('2020-04-01', '2021-03-31'))
                  #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))
                   .map(lambda imag: imag.clip(Geometrysite2)))


satdataset3 = (ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
              .filterBounds (Geometrysite3)
              .filter(ee.Filter.date('2020-04-01', '2021-03-31'))
              .map(addmaskL8sr)
              .map(lambda imag: imag.clip(Geometrysite3)))

landsurfacedata3 = (ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')
                   .filterBounds (Geometrysite3)
                   .filter(ee.Filter.date('2020-04-01', '2021-03-31'))
                  #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))
                   .map(lambda imag: imag.clip(Geometrysite3)))

# Nouvelle section

In [9]:
def surfacedataTOA(image):
  toa = (image.select("B10")).rename('toa_calcul')
  return image.addBands(toa)

In [10]:
def addNDVI(image):
  ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
  return image.addBands(ndvi)

def addNDWI(image):
  ndwi = image.normalizedDifference(['B5', 'B6']).rename('NDWI')
  return image.addBands(ndwi)

def addSAVI(image):
  savi = (((image.select("B5").subtract(image.select("B4"))).divide(image.select("B5").add(image.select("B4")).add(0.5))).multiply(1.5)).rename('SAVI')
  return image.addBands(savi)

In [11]:
def addSWIR2(image):
  swir2 = (image.select("B7")).rename('SWIR2_Band')
  return image.addBands(swir2)

def addSWIR1(image):
  swir1 = (image.select("B6")).rename('SWIR1_Band')
  return image.addBands(swir1)

def addNIR(image):
  nir = (image.select("B5")).rename('NIR_Band')
  return image.addBands(nir)

def addRED(image):
  red = (image.select("B4")).rename('RED_Band')
  return image.addBands(red)

def addGREEN(image):
  green = (image.select("B3")).rename('GREEN_Band')
  return image.addBands(green)

def addBLUE(image):
  blue = (image.select("B2")).rename('BLUE_Band')
  return image.addBands(blue)

In [12]:
SatDataset1 = satdataset1.map(addNDVI).map(addNDWI).map(addSAVI).map(addSWIR2).map(addSWIR1).map(addNIR).map(addRED).map(addGREEN).map(addBLUE)
LandSurfaceDataset1 = landsurfacedata1.map(surfacedataTOA).map(addNDVI)

SatDataset2 = satdataset1.map(addNDVI).map(addNDWI).map(addSAVI).map(addSWIR2).map(addSWIR1).map(addNIR).map(addRED).map(addGREEN).map(addBLUE)
LandSurfaceDataset2 = landsurfacedata2.map(surfacedataTOA).map(addNDVI)

SatDataset3 = satdataset1.map(addNDVI).map(addNDWI).map(addSAVI).map(addSWIR2).map(addSWIR1).map(addNIR).map(addRED).map(addGREEN).map(addBLUE)
LandSurfaceDataset3 = landsurfacedata3.map(surfacedataTOA).map(addNDVI)

In [13]:
TOA_data_Liste1= LandSurfaceDataset1.toList(LandSurfaceDataset1.size())
TOA_data_Liste2= LandSurfaceDataset2.toList(LandSurfaceDataset2.size())
TOA_data_Liste3= LandSurfaceDataset3.toList(LandSurfaceDataset3.size())
print(TOA_data_Liste1.size().getInfo()),
TOA_data_Liste1

23


In [14]:
NDVI_data_Liste1= SatDataset1.toList(SatDataset1.size())

NDVI_data_Liste2= SatDataset2.toList(SatDataset2.size())
NDVI_data_Liste3= SatDataset3.toList(SatDataset3.size())

print(NDVI_data_Liste3.size().getInfo()),
NDVI_data_Liste3

23


In [15]:
NDWI_data_Liste1= SatDataset1.toList(SatDataset1.size())
NDWI_data_Liste2= SatDataset2.toList(SatDataset2.size())
NDWI_data_Liste3= SatDataset3.toList(SatDataset3.size())



In [16]:
SAVI_data_Liste1= SatDataset1.toList(SatDataset1.size())
SWIR2_BAND_Liste1= SatDataset1.toList(SatDataset1.size())
SWIR1_BAND_Liste1= SatDataset1.toList(SatDataset1.size())
NIR_BAND_Liste1= SatDataset1.toList(SatDataset1.size())
RED_BAND_Liste1= SatDataset1.toList(SatDataset1.size())
GREEN_BAND_Liste1= SatDataset1.toList(SatDataset1.size())
BLUE_BAND_Liste1= SatDataset1.toList(SatDataset1.size())

SAVI_data_Liste2= SatDataset2.toList(SatDataset2.size())
SWIR2_BAND_Liste2= SatDataset2.toList(SatDataset2.size())
SWIR1_BAND_Liste2= SatDataset2.toList(SatDataset2.size())
NIR_BAND_Liste2= SatDataset2.toList(SatDataset2.size())
RED_BAND_Liste2= SatDataset2.toList(SatDataset2.size())
GREEN_BAND_Liste2= SatDataset2.toList(SatDataset2.size())
BLUE_BAND_Liste2= SatDataset2.toList(SatDataset2.size())

SAVI_data_Liste3= SatDataset3.toList(SatDataset3.size())
SWIR2_BAND_Liste3= SatDataset3.toList(SatDataset3.size())
SWIR1_BAND_Liste3= SatDataset3.toList(SatDataset3.size())
NIR_BAND_Liste3= SatDataset3.toList(SatDataset3.size())
RED_BAND_Liste3= SatDataset3.toList(SatDataset3.size())
GREEN_BAND_Liste3= SatDataset3.toList(SatDataset3.size())
BLUE_BAND_Liste3= SatDataset3.toList(SatDataset3.size())

In [17]:
TOAListe1 = []
TOAListe2 = []
TOAListe3 = [] 

NDVIListe1 = []
NDVIListe2 = []
NDVIListe3 = []

NDWIListe1 = []
NDWIListe2 = []
NDWIListe3 = []

SAVIListe1 = []
SAVIListe2 = []
SAVIListe3 = []

SWIR2Liste1 = []
SWIR2Liste2 = []
SWIR2Liste3 = []


SWIR1Liste1 = []
SWIR1Liste2 = []
SWIR1Liste3 = []

NIRListe1 = []
NIRListe2 = []
NIRListe3 = []

REDListe1 = []
REDListe2 = []
REDListe3 = []

GREENListe1 =[]
GREENListe2 =[]
GREENListe3 =[]

BLUEListe1 = []
BLUEListe2 = []
BLUEListe3 = []

Date1 = []
Date2 = []
Date3 = []
Hourtime1= []
Hourtime2= []
Hourtime3= []

In [18]:
"""
Surface_TOA_Liste1= Dataset11.toList(Dataset11.size())
TOAListeB10 = []
for i in range(Surface_TOA_Liste1.size().getInfo()):
  image=ee.Image(Surface_TOA_Liste1.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometry1,
      maxPixels=1e15,
      scale=30)
  TOAListeB10.append(np.mean(latitudeLogititude.getInfo().get('toa_calcul')))
"""

"\nSurface_TOA_Liste1= Dataset11.toList(Dataset11.size())\nTOAListeB10 = []\nfor i in range(Surface_TOA_Liste1.size().getInfo()):\n  image=ee.Image(Surface_TOA_Liste1.get(i))\n  latlon_image = ee.Image.pixelLonLat().addBands(image)\n  latitudeLogititude = latlon_image.reduceRegion(\n      reducer=ee.Reducer.toList(),\n      geometry=Geometry1,\n      maxPixels=1e15,\n      scale=30)\n  TOAListeB10.append(np.mean(latitudeLogititude.getInfo().get('toa_calcul')))\n"

In [19]:
"""
TOAListe1 = []
Brightness_Temperature = np.zeros(TOA_data_Liste1.size().getInfo())
for i in range(TOA_data_Liste1.size().getInfo()):
    image=ee.Image(TOA_data_Liste1.get(i))
    latlon_image = ee.Image.pixelLonLat().addBands(image)
    latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometry1,
      maxPixels=1e15,
      scale=30)
    coefficient1 = float(image.getInfo()["properties"]['RADIANCE_ADD_BAND_10'])
    coefficient2 = float(image.getInfo()["properties"]['RADIANCE_MULT_BAND_10'])
    TOA_value = coefficient2 * np.mean(latitudeLogititude.getInfo().get('toa_calcul')) + coefficient1
    K1_value = float(image.getInfo()["properties"]['K1_CONSTANT_BAND_10'])
    K2_value = float(image.getInfo()["properties"]['K2_CONSTANT_BAND_10'])
    Brightness_Temperature[i] = (K2_value / (ln(K1_value/TOA_value) + 1)) - 273.15
    TOAListe1.append(latitudeLogititude.getInfo().get('NDVI'))
NDVI_min=np.array([min(i) for i in TOAListe1])
NDVI_max=np.array([max(i) for i in TOAListe1])
NDVI_mean= np.array([np.mean(i) for i in TOAListe1])
#PV
PV = np.sqrt((NDVI_mean-NDVI_min)/(NDVI_max-NDVI_min))
#epsilom
epsilon=0.004*PV+0.986
# LST
LST = (Brightness_Temperature / ( 1 + (0.00115 * Brightness_Temperature / 1.4388)*ln(epsilon) ) )

"""

'\nTOAListe1 = []\nBrightness_Temperature = np.zeros(TOA_data_Liste1.size().getInfo())\nfor i in range(TOA_data_Liste1.size().getInfo()):\n    image=ee.Image(TOA_data_Liste1.get(i))\n    latlon_image = ee.Image.pixelLonLat().addBands(image)\n    latitudeLogititude = latlon_image.reduceRegion(\n      reducer=ee.Reducer.toList(),\n      geometry=Geometry1,\n      maxPixels=1e15,\n      scale=30)\n    coefficient1 = float(image.getInfo()["properties"][\'RADIANCE_ADD_BAND_10\'])\n    coefficient2 = float(image.getInfo()["properties"][\'RADIANCE_MULT_BAND_10\'])\n    TOA_value = coefficient2 * np.mean(latitudeLogititude.getInfo().get(\'toa_calcul\')) + coefficient1\n    K1_value = float(image.getInfo()["properties"][\'K1_CONSTANT_BAND_10\'])\n    K2_value = float(image.getInfo()["properties"][\'K2_CONSTANT_BAND_10\'])\n    Brightness_Temperature[i] = (K2_value / (ln(K1_value/TOA_value) + 1)) - 273.15\n    TOAListe1.append(latitudeLogititude.getInfo().get(\'NDVI\'))\nNDVI_min=np.array([min(i

LST = (BT / (1 + (0.00115 * BT / 1.4388) * Ln(ε)))



In [20]:
for i in range(NDVI_data_Liste1.size().getInfo()):
  image=ee.Image(NDVI_data_Liste1.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometrysite1,
      maxPixels=1e15,
      scale=30)
  NDVIListe1.append(np.mean(latitudeLogititude.getInfo().get('NDVI')))
  Date1.append(image.getInfo()["properties"]['SENSING_TIME'])

In [21]:
for i in range(NDVI_data_Liste2.size().getInfo()):
  image=ee.Image(NDVI_data_Liste2.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometrysite2,
      maxPixels=1e15,
      scale=30)
  NDVIListe2.append(np.mean(latitudeLogititude.getInfo().get('NDVI')))
  Date2.append(image.getInfo()["properties"]['SENSING_TIME'])

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [22]:
for i in range(NDVI_data_Liste3.size().getInfo()):
  image=ee.Image(NDVI_data_Liste3.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometrysite3,
      maxPixels=1e15,
      scale=30)
  NDVIListe3.append(np.mean(latitudeLogititude.getInfo().get('NDVI')))
  Date3.append(image.getInfo()["properties"]['SENSING_TIME'])

In [23]:
for i in range(NDWI_data_Liste1.size().getInfo()):
  image=ee.Image(NDWI_data_Liste1.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometrysite1,
      maxPixels=1e15,
      scale=30)
  NDWIListe1.append(np.mean(latitudeLogititude.getInfo().get('NDWI')))

In [24]:
for i in range(NDWI_data_Liste2.size().getInfo()):
  image=ee.Image(NDWI_data_Liste2.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometrysite2,
      maxPixels=1e15,
      scale=30)
  NDWIListe2.append(np.mean(latitudeLogititude.getInfo().get('NDWI')))

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [25]:
for i in range(NDWI_data_Liste3.size().getInfo()):
  image=ee.Image(NDWI_data_Liste3.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometrysite3,
      maxPixels=1e15,
      scale=30)
  NDWIListe3.append(np.mean(latitudeLogititude.getInfo().get('NDWI')))

In [26]:
for i in range(SAVI_data_Liste1.size().getInfo()):
  image=ee.Image(SAVI_data_Liste1.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometrysite1,
      maxPixels=1e15,
      scale=30)
  SAVIListe1.append(np.mean(latitudeLogititude.getInfo().get('SAVI')))

In [27]:
for i in range(SAVI_data_Liste2.size().getInfo()):
  image=ee.Image(SAVI_data_Liste2.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometrysite2,
      maxPixels=1e15,
      scale=30)
  SAVIListe2.append(np.mean(latitudeLogititude.getInfo().get('SAVI')))

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [28]:
for i in range(SAVI_data_Liste3.size().getInfo()):
  image=ee.Image(SAVI_data_Liste3.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometrysite3,
      maxPixels=1e15,
      scale=30)
  SAVIListe3.append(np.mean(latitudeLogititude.getInfo().get('SAVI')))

In [29]:
for i in range(SWIR2_BAND_Liste1.size().getInfo()):
  image=ee.Image(SWIR2_BAND_Liste1.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometrysite1,
      maxPixels=1e15,
      scale=30)
  SWIR2Liste1.append(np.mean(latitudeLogititude.getInfo().get('SWIR2_Band')))

In [30]:
for i in range(SWIR2_BAND_Liste2.size().getInfo()):
  image=ee.Image(SWIR2_BAND_Liste2.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometrysite2,
      maxPixels=1e15,
      scale=30)
  SWIR2Liste2.append(np.mean(latitudeLogititude.getInfo().get('SWIR2_Band')))

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [31]:
for i in range(SWIR2_BAND_Liste3.size().getInfo()):
  image=ee.Image(SWIR2_BAND_Liste3.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometrysite3,
      maxPixels=1e15,
      scale=30)
  SWIR2Liste3.append(np.mean(latitudeLogititude.getInfo().get('SWIR2_Band')))

In [32]:
for i in range(SWIR1_BAND_Liste1.size().getInfo()):
  image=ee.Image(SWIR1_BAND_Liste1.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometrysite1,
      maxPixels=1e15,
      scale=30)
  SWIR1Liste1.append(np.mean(latitudeLogititude.getInfo().get('SWIR1_Band')))

In [33]:
for i in range(SWIR1_BAND_Liste2.size().getInfo()):
  image=ee.Image(SWIR1_BAND_Liste2.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometrysite2,
      maxPixels=1e15,
      scale=30)
  SWIR1Liste2.append(np.mean(latitudeLogititude.getInfo().get('SWIR1_Band')))

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [34]:
for i in range(SWIR1_BAND_Liste3.size().getInfo()):
  image=ee.Image(SWIR1_BAND_Liste3.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometrysite3,
      maxPixels=1e15,
      scale=30)
  SWIR1Liste3.append(np.mean(latitudeLogititude.getInfo().get('SWIR1_Band')))

In [35]:
for i in range(NIR_BAND_Liste1.size().getInfo()):
  image=ee.Image(NIR_BAND_Liste1.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometrysite1,
      maxPixels=1e15,
      scale=30)
  NIRListe1.append(np.mean(latitudeLogititude.getInfo().get('NIR_Band')))

In [36]:
for i in range(NIR_BAND_Liste2.size().getInfo()):
  image=ee.Image(NIR_BAND_Liste2.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometrysite2,
      maxPixels=1e15,
      scale=30)
  NIRListe2.append(np.mean(latitudeLogititude.getInfo().get('NIR_Band')))

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [37]:
for i in range(NIR_BAND_Liste3.size().getInfo()):
  image=ee.Image(NIR_BAND_Liste3.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometrysite3,
      maxPixels=1e15,
      scale=30)
  NIRListe3.append(np.mean(latitudeLogititude.getInfo().get('NIR_Band')))

In [38]:
for i in range(RED_BAND_Liste1.size().getInfo()):
  image=ee.Image(RED_BAND_Liste1.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometrysite1,
      maxPixels=1e15,
      scale=30)
  REDListe1.append(np.mean(latitudeLogititude.getInfo().get('RED_Band')))

In [39]:
for i in range(RED_BAND_Liste2.size().getInfo()):
  image=ee.Image(RED_BAND_Liste2.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometrysite2,
      maxPixels=1e15,
      scale=30)
  REDListe2.append(np.mean(latitudeLogititude.getInfo().get('RED_Band')))

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [40]:
for i in range(RED_BAND_Liste3.size().getInfo()):
  image=ee.Image(RED_BAND_Liste3.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometrysite3,
      maxPixels=1e15,
      scale=30)
  REDListe3.append(np.mean(latitudeLogititude.getInfo().get('RED_Band')))

In [41]:
for i in range(GREEN_BAND_Liste1.size().getInfo()):
  image=ee.Image(GREEN_BAND_Liste1.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometrysite1,
      maxPixels=1e15,
      scale=30)
  GREENListe1.append(np.mean(latitudeLogititude.getInfo().get('GREEN_Band')))

In [42]:
for i in range(GREEN_BAND_Liste2.size().getInfo()):
  image=ee.Image(GREEN_BAND_Liste2.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometrysite2,
      maxPixels=1e15,
      scale=30)
  GREENListe2.append(np.mean(latitudeLogititude.getInfo().get('GREEN_Band')))

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [43]:
for i in range(GREEN_BAND_Liste3.size().getInfo()):
  image=ee.Image(GREEN_BAND_Liste3.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometrysite3,
      maxPixels=1e15,
      scale=30)
  GREENListe3.append(np.mean(latitudeLogititude.getInfo().get('GREEN_Band')))

In [44]:
for i in range(BLUE_BAND_Liste1.size().getInfo()):
  image=ee.Image(BLUE_BAND_Liste1.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometrysite1,
      maxPixels=1e15,
      scale=30)
  BLUEListe1.append(np.mean(latitudeLogititude.getInfo().get('BLUE_Band')))

In [45]:
for i in range(BLUE_BAND_Liste2.size().getInfo()):
  image=ee.Image(BLUE_BAND_Liste2.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometrysite2,
      maxPixels=1e15,
      scale=30)
  BLUEListe2.append(np.mean(latitudeLogititude.getInfo().get('BLUE_Band')))

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [46]:
for i in range(BLUE_BAND_Liste3.size().getInfo()):
  image=ee.Image(BLUE_BAND_Liste3.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometrysite3,
      maxPixels=1e15,
      scale=30)
  BLUEListe3.append(np.mean(latitudeLogititude.getInfo().get('BLUE_Band')))

In [47]:
column_names = ['Hourtime', 'Date', 'Air_temperature', 'Soil_temperature', 'Precipitation', 'BLUE_BAND', 'GREEN_BAND', 'RED_BAND', 'NIR_BAND', 'SWIR1_BAND', 'SWIR2_BAND', 'NDVI', 'NDWI', 'SAVI', 'Leaf_Area_index', 'Land_surface_temperature', 'Volumetric_Soil_Moisture']

In [48]:
Datasetintermediairesite1 = pd.DataFrame(columns = column_names)
Datasetintermediairesite2 = pd.DataFrame(columns = column_names)
Datasetintermediairesite3 = pd.DataFrame(columns = column_names)

In [49]:
Date11 = [ x[0:16] for x in Date1 ]
Date11 = [x.replace('T', '') for x in Date11]
Datetime1 = [ x[0:10] for x in Date11 ]
Hourtime1 = [x[10:16] for x in Date11]
Hourtime1 = [ x[:2] + ':' +  '00' for x in Hourtime1 ]

Date22 = [ x[0:16] for x in Date2 ]
Date22 = [x.replace('T', '') for x in Date22]
Datetime2 = [ x[0:10] for x in Date22 ]
Hourtime2 = [x[10:16] for x in Date22]
Hourtime2 = [ x[:2] + ':' +  '00' for x in Hourtime2 ]

Date33 = [ x[0:16] for x in Date3 ]
Date33 = [x.replace('T', '') for x in Date33]
Datetime3 = [ x[0:10] for x in Date33 ]
Hourtime3 = [x[10:16] for x in Date33]
Hourtime3 = [ x[:2] + ':' +  '00' for x in Hourtime3 ]

In [50]:
Datasetintermediairesite1["NDVI"] = NDVIListe1
Datasetintermediairesite1["NDWI"] = NDWIListe1
Datasetintermediairesite1["SAVI"] = SAVIListe1
Datasetintermediairesite1["BLUE_BAND"] = BLUEListe1
Datasetintermediairesite1["GREEN_BAND"] = GREENListe1
Datasetintermediairesite1["RED_BAND"] = REDListe1
Datasetintermediairesite1["NIR_BAND"] = NIRListe1
Datasetintermediairesite1["SWIR1_BAND"] = SWIR1Liste1
Datasetintermediairesite1["SWIR2_BAND"] = SWIR2Liste1
Datasetintermediairesite1["Date"] = Datetime1
Datasetintermediairesite1["Hourtime"] = Hourtime1
Datasetintermediairesite1["Date"] = pd.to_datetime(Datetime1, format='%Y-%m-%d', errors='ignore')

In [51]:
Datasetintermediairesite2["NDVI"] = NDVIListe2
Datasetintermediairesite2["NDWI"] = NDWIListe2
Datasetintermediairesite2["SAVI"] = SAVIListe2
Datasetintermediairesite2["BLUE_BAND"] = BLUEListe2
Datasetintermediairesite2["GREEN_BAND"] = GREENListe2
Datasetintermediairesite2["RED_BAND"] = REDListe2 
Datasetintermediairesite2["NIR_BAND"] = NIRListe2
Datasetintermediairesite2["SWIR1_BAND"] = SWIR1Liste2
Datasetintermediairesite2["SWIR2_BAND"] = SWIR2Liste2
Datasetintermediairesite2["Date"] = Datetime2
Datasetintermediairesite2["Hourtime"] = Hourtime2
Datasetintermediairesite2["Date"] = pd.to_datetime(Datetime2, format='%Y-%m-%d', errors='ignore')

In [52]:
Datasetintermediairesite3["NDVI"] = NDVIListe3
Datasetintermediairesite3["NDWI"] = NDWIListe3
Datasetintermediairesite3["SAVI"] = SAVIListe3
Datasetintermediairesite3["BLUE_BAND"] = BLUEListe3
Datasetintermediairesite3["GREEN_BAND"] = GREENListe3
Datasetintermediairesite3["RED_BAND"] = REDListe3 
Datasetintermediairesite3["NIR_BAND"] = NIRListe3
Datasetintermediairesite3["SWIR1_BAND"] = SWIR1Liste3
Datasetintermediairesite3["SWIR2_BAND"] = SWIR2Liste3
Datasetintermediairesite3["Date"] = Datetime3
Datasetintermediairesite3["Hourtime"] = Hourtime3
Datasetintermediairesite3["Date"] = pd.to_datetime(Datetime3, format='%Y-%m-%d', errors='ignore')

In [ ]:
Datasetintermediairesite1["Leaf_Area_index"] = [2.88 * x + 1.14 * (1 + 0.104 * math.exp(4.1 * x)) for x in NDWIListe1]
Datasetintermediairesite2["Leaf_Area_index"] = [2.88 * x + 1.14 * (1 + 0.104 * math.exp(4.1 * x)) for x in NDWIListe2]
Datasetintermediairesite3["Leaf_Area_index"] = [2.88 * x + 1.14 * (1 + 0.104 * math.exp(4.1 * x)) for x in NDWIListe3]

In [59]:
weatherdata = pd.read_excel('/content/drive/MyDrive/stage_agriedge/weatherdata.xlsx', index_col=0)  

In [61]:
weatherdata.head(2)

,Date time,Maximum Temperature,Minimum Temperature,Temperature,Wind Chill,Heat Index,Precipitation,Snow,Snow Depth,Wind Speed,Wind Direction,Wind Gust,Visibility,Cloud Cover,Relative Humidity,Conditions
Name,,,,,,,,,,,,,,,,
"بن جرير, مراكش - آسفي, المغرب",04/01/2020 00:00:00,15.0,15.0,15.0,NaN,NaN,0.0,NaN,NaN,13.0,270.0,NaN,8.0,89.0,72.03,Overcast
"بن جرير, مراكش - آسفي, المغرب",04/01/2020 01:00:00,13.9,13.9,13.9,NaN,NaN,0.0,NaN,NaN,15.9,275.0,NaN,8.0,74.3,81.33,Partially cloudy


In [62]:
date2 = []
hourtime2 = []
date2 = [x[:10] for x in weatherdata["Date time"]]
hourtime2 = [x[11:] for x in weatherdata["Date time"]]
date2 = [x.replace('/', '-') for x in date2]
date2 = [ x[6:] + '-' + x[:2] + '-' + x[3:5] for x in date2 ]
weatherdata["Date"] = pd.to_datetime(date2, format='%Y-%m-%d', errors='ignore')
weatherdata['Hour'] = hourtime2

In [63]:
weatherdata = weatherdata.drop(columns='Date time', axis = 1)
weatherdata.head(2)

,Maximum Temperature,Minimum Temperature,Temperature,Wind Chill,Heat Index,Precipitation,Snow,Snow Depth,Wind Speed,Wind Direction,Wind Gust,Visibility,Cloud Cover,Relative Humidity,Conditions,Date,Hour
Name,,,,,,,,,,,,,,,,,
"بن جرير, مراكش - آسفي, المغرب",15.0,15.0,15.0,NaN,NaN,0.0,NaN,NaN,13.0,270.0,NaN,8.0,89.0,72.03,Overcast,2020-04-01,00:00:00
"بن جرير, مراكش - آسفي, المغرب",13.9,13.9,13.9,NaN,NaN,0.0,NaN,NaN,15.9,275.0,NaN,8.0,74.3,81.33,Partially cloudy,2020-04-01,01:00:00


In [64]:
Datasetintermediairesite11 = pd.merge(Datasetintermediairesite1,weatherdata[weatherdata["Hour"]=="11:00:00"][["Temperature", "Precipitation", "Date"]], on='Date',suffixes=('_dataset', ''))
Datasetintermediairesite22 = pd.merge(Datasetintermediairesite2,weatherdata[weatherdata["Hour"]=="11:00:00"][["Temperature", "Precipitation", "Date"]], on='Date',suffixes=('_dataset', ''))
Datasetintermediairesite33 = pd.merge(Datasetintermediairesite3,weatherdata[weatherdata["Hour"]=="11:00:00"][["Temperature", "Precipitation", "Date"]], on='Date',suffixes=('_dataset', ''))

In [69]:
Datasetintermediairesite33.head(3)

,Hourtime,Date,Soil_temperature,BLUE_BAND,GREEN_BAND,RED_BAND,NIR_BAND,SWIR1_BAND,SWIR2_BAND,NDVI,NDWI,SAVI,Leaf_Area_index,Land_surface_temperature,Volumetric_Soil_Moisture,Temperature,Precipitation
0,11:00,2020-04-06,NaN,468.5,834.0,1189.5,1840.0,1789.5,1281.0,0.215909,0.016975,0.323809,1.315994,NaN,NaN,21.0,0.0
1,11:00,2020-04-22,NaN,844.0,1349.0,1939.5,3154.0,3657.0,2943.0,0.238484,-0.073828,0.357691,1.014971,NaN,NaN,19.0,0.0
2,11:00,2020-05-08,NaN,841.5,1355.5,1887.5,3087.5,3625.5,2966.0,0.241206,-0.080141,0.361773,0.994552,NaN,NaN,22.0,0.0


In [67]:
Datasetintermediairesite11 = Datasetintermediairesite11.drop(["Air_temperature", "Precipitation_dataset"], axis = 1)
Datasetintermediairesite22 = Datasetintermediairesite22.drop(["Air_temperature", "Precipitation_dataset"], axis = 1)
Datasetintermediairesite33 = Datasetintermediairesite33.drop(["Air_temperature", "Precipitation_dataset"], axis = 1)

In [68]:
Datasetintermediairesite11.head(3)

,Hourtime,Date,Soil_temperature,BLUE_BAND,GREEN_BAND,RED_BAND,NIR_BAND,SWIR1_BAND,SWIR2_BAND,NDVI,NDWI,SAVI,Leaf_Area_index,Land_surface_temperature,Volumetric_Soil_Moisture,Temperature,Precipitation
0,11:00,2020-04-06,NaN,254.8,521.8,671.4,1244.6,1029.0,690.6,0.299333,0.094882,0.448880,1.588200,NaN,NaN,21.0,0.0
1,11:00,2020-04-22,NaN,771.0,1267.2,1667.2,3396.0,3404.4,2627.6,0.342002,-0.000574,0.512952,1.256629,NaN,NaN,19.0,0.0
2,11:00,2020-05-08,NaN,763.6,1268.0,1681.6,3206.2,3280.6,2521.4,0.312727,-0.010980,0.469042,1.221720,NaN,NaN,22.0,0.0


In [70]:
capteurdata11 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site1/mars.csv", sep=";")
capteurdata12 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site1/mai.csv", sep=";")
capteurdata13 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site1/juin.csv", sep=";")
capteurdata14 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site1/juillet.csv", sep=";")
capteurdata15 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site1/aout.csv", sep=";")
capteurdata16 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site1/septembre.csv", sep=";")
capteurdata17 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site1/octobre.csv", sep=";")
capteurdata18 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site1/novembre.csv", sep=";")
capteurdata19 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site1/decembre.csv", sep=";")
capteurdata110 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site1/janvier2021.csv", sep=";")
capteurdata111 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site1/fevrier2021.csv", sep=";")

In [ ]:
capteurdata21 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site2/mars.csv", sep=";")
capteurdata22 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site2/mai.csv", sep=";")
capteurdata23 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site2/juin.csv", sep=";")
capteurdata24 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site2/juillet.csv", sep=";")
capteurdata25 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site2/aout.csv", sep=";")
capteurdata26 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site2/septembre.csv", sep=";")
capteurdata27 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site2/octobre.csv", sep=";")
capteurdata28 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site2/novembre.csv", sep=";")
capteurdata29 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site2/decembre.csv", sep=";")
capteurdata210 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site2/janvier2021.csv", sep=";")
capteurdata211 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site2/fevrier2021.csv", sep=";")

In [ ]:
capteurdata31 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site3/mars.csv", sep=";")
capteurdata32 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site3/mai.csv", sep=";")
capteurdata33 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site3/juin.csv", sep=";")
capteurdata34 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site3/juillet.csv", sep=";")
capteurdata35 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site3/aout.csv", sep=";")
capteurdata36 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site3/septembre.csv", sep=";")
capteurdata37 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site3/octobre.csv", sep=";")
capteurdata38 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site3/novembre.csv", sep=";")
capteurdata39 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site3/decembre.csv", sep=";")
capteurdata310 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site3/janvier2021.csv", sep=";")
capteurdata311 = pd.read_csv("/content/drive/MyDrive/stage_agriedge/site3/fevrier2021.csv", sep=";")

In [ ]:
capteurdata11["rpi_timestamp"] = [x.replace('T', '') for x in  capteurdata11["rpi_timestamp"]]
capteurdata11["Date"] = [ x[0:10] for x in capteurdata11["rpi_timestamp"] ]
capteurdata11["Hour"] = [ x[10:15] for x in capteurdata11["rpi_timestamp"] ]
capteurdata11.drop(["rpi_timestamp"], axis = 1)

capteurdata12["rpi_timestamp"] = [x.replace('T', '') for x in  capteurdata12["rpi_timestamp"]]
capteurdata12["Date"] = [ x[0:10] for x in capteurdata12["rpi_timestamp"] ]
capteurdata12["Hour"] = [ x[10:15] for x in capteurdata12["rpi_timestamp"] ]
capteurdata12.drop(["rpi_timestamp"], axis = 1)

capteurdata13["rpi_timestamp"] = [x.replace('T', '') for x in  capteurdata13["rpi_timestamp"]]
capteurdata13["Date"] = [ x[0:10] for x in capteurdata13["rpi_timestamp"] ]
capteurdata13["Hour"] = [ x[10:15] for x in capteurdata13["rpi_timestamp"] ]
capteurdata13.drop(["rpi_timestamp"], axis = 1)

capteurdata14["rpi_timestamp"] = [x.replace('T', '') for x in  capteurdata14["rpi_timestamp"]]
capteurdata14["Date"] = [ x[0:10] for x in capteurdata14["rpi_timestamp"] ]
capteurdata14["Hour"] = [ x[10:15] for x in capteurdata14["rpi_timestamp"] ]
capteurdata14.drop(["rpi_timestamp"], axis = 1)

capteurdata15["rpi_timestamp"] = [x.replace('T', '') for x in  capteurdata15["rpi_timestamp"]]
capteurdata15["Date"] = [ x[0:10] for x in capteurdata15["rpi_timestamp"] ]
capteurdata15["Hour"] = [ x[10:15] for x in capteurdata15["rpi_timestamp"] ]
capteurdata15.drop(["rpi_timestamp"], axis = 1)

capteurdata16["rpi_timestamp"] = [x.replace('T', '') for x in  capteurdata16["rpi_timestamp"]]
capteurdata16["Date"] = [ x[0:10] for x in capteurdata16["rpi_timestamp"] ]
capteurdata16["Hour"] = [ x[10:15] for x in capteurdata16["rpi_timestamp"] ]
capteurdata16.drop(["rpi_timestamp"], axis = 1)

capteurdata17["rpi_timestamp"] = [x.replace('T', '') for x in  capteurdata17["rpi_timestamp"]]
capteurdata17["Date"] = [ x[0:10] for x in capteurdata17["rpi_timestamp"] ]
capteurdata17["Hour"] = [ x[10:15] for x in capteurdata17["rpi_timestamp"] ]
capteurdata17.drop(["rpi_timestamp"], axis = 1)

capteurdata18["rpi_timestamp"] = [x.replace('T', '') for x in  capteurdata18["rpi_timestamp"]]
capteurdata18["Date"] = [ x[0:10] for x in capteurdata18["rpi_timestamp"] ]
capteurdata18["Hour"] = [ x[10:15] for x in capteurdata18["rpi_timestamp"] ]
capteurdata18.drop(["rpi_timestamp"], axis = 1)

capteurdata19["rpi_timestamp"] = [x.replace('T', '') for x in  capteurdata19["rpi_timestamp"]]
capteurdata19["Date"] = [ x[0:10] for x in capteurdata19["rpi_timestamp"] ]
capteurdata19["Hour"] = [ x[10:15] for x in capteurdata19["rpi_timestamp"] ]
capteurdata19.drop(["rpi_timestamp"], axis = 1)

capteurdata110["rpi_timestamp"] = [x.replace('T', '') for x in  capteurdata110["rpi_timestamp"]]
capteurdata110["Date"] = [ x[0:10] for x in capteurdata110["rpi_timestamp"] ]
capteurdata110["Hour"] = [ x[10:15] for x in capteurdata110["rpi_timestamp"] ]
capteurdata110.drop(["rpi_timestamp"], axis = 1)

capteurdata111["rpi_timestamp"] = [x.replace('T', '') for x in  capteurdata111["rpi_timestamp"]]
capteurdata111["Date"] = [ x[0:10] for x in capteurdata111["rpi_timestamp"] ]
capteurdata111["Hour"] = [ x[10:15] for x in capteurdata111["rpi_timestamp"] ]
capteurdata111.drop(["rpi_timestamp"], axis = 1)

In [ ]:
datasetcapteursite1 = pd.concat([capteurdata11,capteurdata12, capteurdata13, capteurdata14, capteurdata15, capteurdata16, 
                            capteurdata17, capteurdata18, capteurdata19, capteurdata110, capteurdata111],ignore_index=True)

In [ ]:
datasetcapteursite2 = pd.concat([capteurdata21,capteurdata22, capteurdata23, capteurdata24, capteurdata25, capteurdata26, 
                            capteurdata27, capteurdata28, capteurdata29, capteurdata210, capteurdata211],ignore_index=True)

In [ ]:
datasetcapteursite3 = pd.concat([capteurdata31,capteurdata32, capteurdata33, capteurdata34, capteurdata35, capteurdata36, 
                            capteurdata37, capteurdata38, capteurdata39, capteurdata310, capteurdata311],ignore_index=True)

In [ ]:
Date1 = datasetcapteur["Date"]
datasetcapteur["Date"] = pd.to_datetime(Date1, format='%Y-%m-%d', errors='ignore')
new_data_regression = pd.merge(new_data, datasetcapteur[datasetcapteur["Hour"]=="11:00"][["soil_temperature", "Date"]], on='Date',suffixes=('_dataset', ''))

In [ ]:
new_data_regression.head()

In [ ]:

y = target
X = diabetes
# setting up testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)